In [ ]:
import json
from typing import List
import requests

def brave_search(api_key: str, query: str, search_kwargs: dict = {}) -> List[List[str]]:
    base_url = "https://api.search.brave.com/res/v1/web/search"
    
    def _search_request(query: str) -> List[dict]:
        headers = {
            "X-Subscription-Token": api_key,
            "Accept": "application/json",
        }
        req = requests.PreparedRequest()
        params = {**search_kwargs, **{"q": query, "extra_snippets": True}}
        req.prepare_url(base_url, params)
        if req.url is None:
            raise ValueError("prepared url is None, this should not happen")

        response = requests.get(req.url, headers=headers)
        if not response.ok:
            raise Exception(f"HTTP error {response.status_code}")

        return response.json().get("web", {}).get("results", [])

    web_search_results = _search_request(query=query)
    final_results = [
        [
            " ".join(
                filter(
                    None, [item.get("description"), *item.get("extra_snippets", [])]
                )
            ),
            item.get("title"),
            item.get("url"),
        ]
        for item in web_search_results
    ]
    return final_results

# Example usage:
api_key = "your_api_key_here"
query = "example search"
results = brave_search(api_key, query)

for result in results:
    print(f"Page Content: {result[0]}")
    print(f"Title: {result[1]}")
    print(f"Link: {result[2]}")
    print()